In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import read_file as rf
import split_data as spt

# Latent factor-based  подход

В данном подходе мы вводим латентные переменные, чтобы выявить зависимость на основе оценок между пользователями и фильмами. При введении латентных переменных мы можем получить две матрицы: одна описывает пользователей в пространстве мкрытых переменных, а вторая описывает фильмы в этом пространстве. Если перемножить их, то получим значения оценок пользователей фильмам.

Иными словами, оценка r<sub>ui</sub> пользователя u, поставленная фильму i, ищется как скалярное произведение векторов p<sub>u</sub> и q<sub>i</sub> в некотором пространстве латентных признаков размерности K:

$$\hat{r}_{u, i} = p_{u}^T q_{i}$$

Для решения данной задачи будем минимизировать следующий функционал:

$$\sum_{(u, i, r_{u,i})} (r_{u, i} - p_{u}^Tq_{i})^2 + \lambda_{p}p_{u}^Tp_{u} + \lambda_{q}q_{i}^Tq_{i}$$

Суммирование ведется по всем тройкам  (u, i, r<sub>u,i</sub>) a слагаемые с лямбдой добавлены для регуляризации

Воспользуемся следующим методом для оптимизации данного функционала. Составим матрицу P из векторов p<sub>u</sub> и матрицу Q из векторов q<sub>i</sub>. Будем делать последовательные шаги и минимизировать каждую из матриц при фиксированной второй. Опишим эти шаги:
- Шаг перенастройки матрицы P:
    $$A_{u} = Q[u]^T Q[u]$$
    $$d_{u} = Q[u]^T r_{u} $$
    $$p_{u} = (\lambda_{p}n_{u}I + A_{u})^{-1}d_{u}$$
- Шаг перенастройки матрицы Q:
    $$B_{i} = P[i]^T P[i]$$
    $$c_{i} = P[i]^T r_{i} $$
    $$q_{i} = (\lambda_{q}n_{i}I + B_{i})^{-1}c_{i}$$

где n<sub>u</sub> -- количество оценок пользователя u и n<sub>i</sub> -- количество оценок фильма i

### Считываем данные

In [2]:
df_users, df_movies, df_rates = rf.read_zipped_file('ml-1m.zip')

### Делим данные

In [3]:
%time df_splited = spt.get_train_test_split(df_rates, df_users.index)

CPU times: user 55.6 s, sys: 50.4 s, total: 1min 46s
Wall time: 1min 47s


In [4]:
%%time
movies_ind = spt.get_new_indexes(df_splited, 'movie_id')
users_ind = spt.get_new_indexes(df_splited, 'user_id')

CPU times: user 313 ms, sys: 7.59 ms, total: 320 ms
Wall time: 324 ms


###  Используем Ridge-регрессию

In [13]:
df = df_splited[0].copy()

In [26]:
%%time

def evaluate_matrix(N, P, Q, lambda_p, lambda_q):
    for i in range(N):
        for u in users_ind[0]:
            Q_u = Q[users_movies[u]-1, :]
            A_u = np.dot(Q_u.T, Q_u)
            d_u = np.dot(Q_u.T, rates_users[u])
            P[u-1, :] = np.dot(np.linalg.inv((lambda_p*len(rates_users[u])*np.eye(A_u.shape[0]) + A_u)), d_u)
        for i in movies_ind[0]:
            P_i = P[movies_users[i]-1, :]
            A_i = np.dot(P_i.T, P_i)
            d_i = np.dot(P_i.T, rates_movies[i])
            Q[i-1, :] = np.dot(np.linalg.inv((lambda_q*len(rates_movies[i])*np.eye(A_i.shape[0]) + A_i)), d_i)
    print ('loss = ', mse(np.dot(P, Q.T), df_splited[1]))

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.06 µs


In [23]:
%%time
def mse (pred, df):
    r_pred = pred.copy()
    y_pred = []
    y_true = []
    for ind in df.index:
        row = df.loc[ind]
        y_true.append(row['rating'])
        user = row['user_id']
        movie = row['movie_id']
        y_pred.append(r_pred[user-1][movie-1])
    loss = mean_squared_error(y_true=y_true, y_pred=y_pred)
    return loss

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 9.06 µs


In [10]:
%%time

df_grouped =  df.groupby(df.user_id)
users_movies = {}
rates_users = {}
for group in df_grouped.groups:
    users_movies[group] = df_grouped.get_group(group).movie_id.as_matrix()
    rates_users[group] = df_grouped.get_group(group).rating.as_matrix()
    
df_grouped =  df.groupby(df.movie_id)
movies_users = {}
rates_movies = {}
for group in df_grouped.groups:
    movies_users[group] = df_grouped.get_group(group).user_id.as_matrix()
    rates_movies[group] = df_grouped.get_group(group).rating.as_matrix()

CPU times: user 7.17 s, sys: 207 ms, total: 7.38 s
Wall time: 7.47 s


Инициализируем матрицы

Подберем параметры на отложенной выборке. 

In [19]:
lambda_p = 0.2
lambda_q = 0.001
N = 20
K = 10
P = 0.1 * np.random.random((df_users.index.max(), K)) 
Q = 0.1 * np.random.random((df_movies.index.max(), K))

In [24]:
l_p = [0.001, 0.01, 0.1, 1]
l_q = [0.001, 0.01, 0.1, 1]
for p in l_p:
    for q in l_q:
        print (' p = %f, q = %f' %(p, q))
        N = 1
        K = 10
        P = p * np.random.random((df_users.index.max(), K)) 
        Q = q * np.random.random((df_movies.index.max(), K))
        evaluate_matrix(N, P, Q, lambda_p, lambda_q)

 p = 0.001000, q = 0.001000
loss =  4.44320468657
 p = 0.001000, q = 0.010000
loss =  0.892360675494
 p = 0.001000, q = 0.100000
loss =  0.902871389021
 p = 0.001000, q = 1.000000
loss =  0.956106863014
 p = 0.010000, q = 0.001000
loss =  4.50143527211
 p = 0.010000, q = 0.010000
loss =  0.890632209347
 p = 0.010000, q = 0.100000
loss =  0.902536362368
 p = 0.010000, q = 1.000000
loss =  0.952564209634
 p = 0.100000, q = 0.001000
loss =  4.46224786628
 p = 0.100000, q = 0.010000
loss =  0.89592198435
 p = 0.100000, q = 0.100000
loss =  0.905623880018
 p = 0.100000, q = 1.000000
loss =  0.955978480751
 p = 1.000000, q = 0.001000
loss =  4.40881936501
 p = 1.000000, q = 0.010000
loss =  0.888655335624
 p = 1.000000, q = 0.100000
loss =  0.896975619487
 p = 1.000000, q = 1.000000
loss =  0.961075982937


Видим, что лучший результат при p=1 и q=0.01

In [26]:
l_p = [0.001, 0.01, 0.1, 1]
l_q = [0.001, 0.01, 0.1, 1]
for p in l_p:
    for q in l_q:
        print (' p = %f, q = %f' %(p, q))
        N = 1
        K = 10
        P = 1 * np.random.random((df_users.index.max(), K)) 
        Q = 0.01 * np.random.random((df_movies.index.max(), K))
        evaluate_matrix(N, P, Q, p, q)

 p = 0.001000, q = 0.001000
loss =  1.01741830304
 p = 0.001000, q = 0.010000
loss =  0.975822732789
 p = 0.001000, q = 0.100000
loss =  0.922147330781
 p = 0.001000, q = 1.000000
loss =  0.893799430423
 p = 0.010000, q = 0.001000
loss =  0.923468242771
 p = 0.010000, q = 0.010000
loss =  0.893391976445
 p = 0.010000, q = 0.100000
loss =  0.891775677089
 p = 0.010000, q = 1.000000
loss =  0.890840395604
 p = 0.100000, q = 0.001000
loss =  0.889481099413
 p = 0.100000, q = 0.010000
loss =  0.896228040097
 p = 0.100000, q = 0.100000
loss =  1.47502844884
 p = 0.100000, q = 1.000000
loss =  7.67219007731
 p = 1.000000, q = 0.001000
loss =  1.46523594788
 p = 1.000000, q = 0.010000
loss =  7.6643311513
 p = 1.000000, q = 0.100000
loss =  12.3436294404
 p = 1.000000, q = 1.000000
loss =  13.0504403325


Далее, видим, что наилучшие значения для P = 0.1 и для Q = 0.1

In [27]:
k = [5, 10, 15, 20]
for K in k:
    print (' k = %d' %K)
    P = 0.1 * np.random.random((df_users.index.max(), K)) 
    Q = 0.1 * np.random.random((df_movies.index.max(), K))
    evaluate_matrix(N, P, Q, p, q)

 k = 5
loss =  0.779247663865
 k = 10
loss =  0.765251867855
 k = 15
loss =  0.762434914081
 k = 20
loss =  0.761164021313


Посмотрим как зависит от размерности пространства скрытых переменных

| K size  | MSE  |
|---|---|
| 5  | 0.779247663865  |
|  10 |  0.765251867855 |
|  100 | 0.762434914081  |
|20|0.761164021313|

Результаты были подсчитаны при N = 20

Результат можно еще улучшить увеличением N до 30

### Выводы

Latent-factor оказался наилучщим методом, он смог достичь наименьшего mse. Очень удобный в реализации метод.
Достоинства:
    - Прост в реализации
    - Достигает хорошего качества при подборе параметров
Недостатки
    - Требует много места для хранения матриц